In [1]:
spark

# join subscription periods data with train.csv

In [2]:
# read member subscription periods
dir0 = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/'
infile = dir0 + '03.subscription_periods'

df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [3]:
df0.show(5)

+--------------------+------------+----------+----------+
|                msno|         tid|start_date|  end_date|
+--------------------+------------+----------+----------+
|a+y7kt+stTydU2fG6...| 68719619252|2016-01-18|2016-02-16|
|a+y7kt+stTydU2fG6...| 94490046943|2016-04-26|2016-11-06|
|a+y7kt+stTydU2fG6...|      205287|2016-03-22|2016-04-20|
|a+y7kt+stTydU2fG6...|154619153540|2016-02-19|2016-03-19|
|a+y7kt+stTydU2fG6...|154619330779|2015-12-19|2016-01-17|
+--------------------+------------+----------+----------+
only showing top 5 rows



In [4]:
df0.count()

14074661

In [15]:
# read train.csv
infile = 'file:///home/cloudera/2.kkbox_churn/raw_data/train.csv'
df_train = spark.read.format('csv').option('header','true').load(infile) \
           .withColumnRenamed('msno', 'msno_')

df_train.printSchema()

root
 |-- msno_: string (nullable = true)
 |-- is_churn: string (nullable = true)



In [17]:
# join
df1 = df0.join(df_train, df0['msno']==df_train['msno_'], 'left_semi') \

df1.printSchema()

root
 |-- msno: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [18]:
# output
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/train-v1/' \
          + '00.join_w_train-v1'
df1.write.format('csv').option('header', 'true').save(outfile)

# remove members with indeterminate dates after 2016-10-01

In [3]:
infile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/train-v1/' \
          + '00.join_w_train-v1'
df0 = spark.read.format('csv').option('header', 'true').load(infile)

df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [4]:
df0.show(5)

+--------------------+-----------+----------+----------+
|                msno|        tid|start_date|  end_date|
+--------------------+-----------+----------+----------+
|+/cL4huZiFBaGK1L8...|68720055399|2017-01-13|2017-02-11|
|+/qMNBKquBhhNGFXx...|42950301900|2016-04-19|2016-05-18|
|+/qMNBKquBhhNGFXx...|42950332288|2016-03-14|2016-04-12|
|+/qMNBKquBhhNGFXx...|17180350476|2016-09-26|2016-10-25|
|+/qMNBKquBhhNGFXx...|25769973191|2017-01-20|2017-02-18|
+--------------------+-----------+----------+----------+
only showing top 5 rows



In [15]:
df0.groupBy('msno').count().count()

# number of members

982738

In [21]:
from pyspark.sql.functions import col
df0.where(col('tid') == 'indeterm').groupBy('msno').count().count()

# number of members with indeterminate periods:

75165

In [22]:
df0.where(col('tid') == 'indeterm').show(10)

+--------------------+--------+----------+----------+
|                msno|     tid|start_date|  end_date|
+--------------------+--------+----------+----------+
|+4R20mT6lKwbCATc9...|indeterm|2016-04-16|2016-05-14|
|+bP6pPqRBihraeu3X...|indeterm|2016-10-22|2016-11-20|
|+k0ZetYc7RxKdDZkU...|indeterm|2016-02-20|2016-03-18|
|+u+YLmXNpOemBT6MM...|indeterm|2016-02-27|2016-03-25|
|/+29ci+SsCCLqtF4G...|indeterm|2015-11-17|2015-12-15|
|/4qXx+zX39HJPoQSu...|indeterm|2016-05-10|2016-06-08|
|/AXw250p2yAL47E1i...|indeterm|2016-03-03|2016-04-01|
|/CjLSU6Ch8swdfmNP...|indeterm|2017-02-11|2017-03-12|
|/cWgRAgN9Id6NSp1+...|indeterm|2016-02-15|2016-03-14|
|/mYRQtb1K3fZHFMTY...|indeterm|2016-12-04|2017-01-02|
+--------------------+--------+----------+----------+
only showing top 10 rows



In [24]:
df1 = df0.where(col('tid') == 'indeterm') \
         .where(col('end_date') >= '2016-10-01')

df1.show(10)

+--------------------+--------+----------+----------+
|                msno|     tid|start_date|  end_date|
+--------------------+--------+----------+----------+
|+bP6pPqRBihraeu3X...|indeterm|2016-10-22|2016-11-20|
|/CjLSU6Ch8swdfmNP...|indeterm|2017-02-11|2017-03-12|
|/mYRQtb1K3fZHFMTY...|indeterm|2016-12-04|2017-01-02|
|/wM07qLyO7cwMUaWy...|indeterm|2017-01-26|2017-02-24|
|0i7ymbzIoucqr1kPH...|indeterm|2016-12-04|2017-01-02|
|0q8VIUHRRHJx+b6uZ...|indeterm|2017-03-01|2017-03-27|
|0rIE22z5+CgZydo4a...|indeterm|2017-01-19|2017-02-17|
|17UUnztZTmgkctgaq...|indeterm|2017-02-25|2017-03-23|
|1CBwn13dbfQbSElCe...|indeterm|2016-11-28|2016-12-26|
|25xGzZM7FSaV/PwYN...|indeterm|2016-11-13|2016-12-12|
+--------------------+--------+----------+----------+
only showing top 10 rows



In [27]:
df2 = df1.groupBy('msno').count()

df2.count()
# number of memebers with indeterminate dates after 2016-10-01

31737

In [28]:
df2.show(2)

+--------------------+-----+
|                msno|count|
+--------------------+-----+
|+0l/WkoOIugT69NYa...|    1|
|+2eLsQv6T46iKwO+m...|    1|
+--------------------+-----+
only showing top 2 rows



In [31]:
# left anti-join
df3 = df0.join(df2, df0['msno']==df2['msno'], 'left_anti')
df3.printSchema()

root
 |-- msno: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [32]:
# output
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/train-v1/' \
          + '01.w-members-no-indeterm-after-2016-Oct'
df3.write.format('csv').option('header','true').save(outfile)

#  remove members with latest end_date < 2017-01-01

In [2]:
# read
infile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/train-v1/' \
          + '01.w-members-no-indeterm-after-2016-Oct'
df0 = spark.read.format('csv').option('header','true').load(infile)

df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [5]:
df0.count()

11061697

In [7]:
df0.groupBy('msno').count().count()

951001

In [3]:
df0.show(3)

+--------------------+-----------+----------+----------+
|                msno|        tid|start_date|  end_date|
+--------------------+-----------+----------+----------+
|+/cL4huZiFBaGK1L8...|68720055399|2017-01-13|2017-02-11|
|+/qMNBKquBhhNGFXx...|42950301900|2016-04-19|2016-05-18|
|+/qMNBKquBhhNGFXx...|42950332288|2016-03-14|2016-04-12|
+--------------------+-----------+----------+----------+
only showing top 3 rows



In [4]:
from pyspark.sql.functions import col, max
df1 = df0.groupBy('msno').agg(max('start_date').alias('max_start_date'), 
                              max('end_date').alias('max_end_date'))

df1.show(10)

+--------------------+--------------+------------+
|                msno|max_start_date|max_end_date|
+--------------------+--------------+------------+
|++4RuqBw0Ss6bQU4o...|    2017-02-13|  2017-03-13|
|+/HS8LzrRGXolKbxR...|    2017-02-17|  2017-03-16|
|+/namlXq+u3izRjHC...|    2017-03-02|  2017-03-31|
|+0/X9tkmyHyet9X80...|    2017-03-02|  2017-03-31|
|+09YGn842g6h2EZUX...|    2017-03-02|  2017-03-31|
|+0RJtbyhoPAHPa+34...|    2016-01-15|  2017-02-27|
|+0jTOa6KGPk1vtNTw...|    2017-02-02|  2017-03-03|
|+0l+FDuhyjaZnu0AP...|    2017-02-24|  2017-03-23|
|+2Df04hr61UUJijMM...|    2017-02-03|  2017-03-04|
|+2oK/qWmYvAnfNZsV...|    2017-02-19|  2017-03-19|
+--------------------+--------------+------------+
only showing top 10 rows



In [10]:
df2 = df1.where(col('max_end_date') < '2017-01-01')
df2.count()

# number of members with latest end_date < 2017-01-01

2489

In [11]:
df2.show(10)

+--------------------+--------------+------------+
|                msno|max_start_date|max_end_date|
+--------------------+--------------+------------+
|+QaT3MJmiHMzJDclo...|    2016-01-07|  2016-02-01|
|1+TE6mn7v3BLe7R0U...|    2016-03-19|  2016-03-25|
|3CvlTvY/XJD3j1ZDm...|    2016-01-06|  2016-01-12|
|8AxKPxTjFbtxA6aYW...|    2016-03-19|  2016-03-25|
|8fop35qcweVLVyCfg...|    2016-11-28|  2016-12-04|
|BzkJZYf3NkXqYWjW6...|    2016-03-11|  2016-04-09|
|KV7GNLqeyhZJ0fJE2...|    2016-11-15|  2016-12-14|
|LbkLV5Ceg1VuEyuiV...|    2016-05-02|  2016-05-31|
|VFvvE/dS6tlzw53YW...|    2016-07-13|  2016-07-20|
|VLXuKoDHSlp2ukEjj...|    2016-04-15|  2016-05-11|
+--------------------+--------------+------------+
only showing top 10 rows



In [13]:
# anti-join
df3 = df0.join(df2, df0['msno']==df2['msno'], 'left_anti')
df3.printSchema()

root
 |-- msno: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [14]:
# output
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/train-v1/' \
          + '02.w-members-latest-end-date-in-2017'
df3.write.format('csv').option('header','true').save(outfile)

# filer: end_date >= 2016-10-01

In [2]:
infile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/train-v1/' \
          + '02.w-members-latest-end-date-in-2017'
df0 = spark.read.format('csv').option('header','true').load(infile)
df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- tid: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)



In [3]:
print(df0.count())
print(df0.groupBy('msno').count().count())

11052785
948512


In [4]:
from pyspark.sql.functions import col
df1 = df0.where(col('end_date') >= '2016-10-01')

In [5]:
print(df1.count())
print(df1.groupBy('msno').count().count())

5119048
948512


In [6]:
# check
df1.where(col('tid') == 'indeterm').count()  # should be 0

0

In [8]:
# output
outfile = 'file:///home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/train-v1/' \
          + '03.members-w-last-end-in-2017_ends-after-20161001'
df1.write.format('csv').option('header','true').save(outfile)